In [12]:
!pip install langchain langgraph openai faiss-cpu sentence-transformers pandas scikit-learn tiktoken


     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ----------------------------------- -- 41.0/44.4 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 44.4/44.4 kB 1.1 MB/s eta 0:00:00
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/145.0 kB ? eta -:--:--
   ------------------------- -------------- 92.2/145.0 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 145.0/145.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
   ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/47.2 kB ? eta -:--:--
   ---------------------------------------- 47.2/47.2 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 125.4/125.4 kB 7.7 MB/s eta 0:00:00
Using cached r

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyrebase 3.0.27 requires requests==2.11.1, but you have requests 2.32.3 which is incompatible.
pyrebase 3.0.27 requires requests-toolbelt==0.7.0, but you have requests-toolbelt 1.0.0 which is incompatible.
unstructured-client 0.28.1 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
unstructured-client 0.28.1 requires pydantic<2.10.0,>=2.9.2, but you have pydantic 2.10.3 which is incompatible.


# Loading the Dataset and Creating an Dataframe

In [ ]:
import pandas as pd

df = pd.read_json("Dataset\News_Category_Dataset_v3.json", lines=True)
df['text'] = df['headline'] + ". " + df['short_description']
df = df[['category', 'text']]
df.dropna(inplace=True)
documents = df['text'].tolist()


First 5 records:
                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until

In [ ]:
from langchain.schema import Document

lc_documents = [Document(page_content=text, metadata={"category": cat}) for text, cat in zip(df['text'], df['category'])]


Batches:   0%|          | 0/6548 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vectorstore = FAISS.from_documents(lc_documents, embedding_model)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [10]:
df

,link,headline,category,short_description,authors,date,combined_text
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...
...,...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28,Maria Sharapova Stunned By Victoria Azarenka I...
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28,"Giants Over Patriots, Jets Over Colts Among M..."
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28,Aldon Smith Arrested: 49ers Linebacker Busted ...
